# Prueba de separación y MSE

## Imports

In [48]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error


## Generacion de un dataset de ejemplo (actualizar para que use el de la tarea)

In [49]:
np.random.seed(0)
X = np.random.rand(1000, 9)     # 1000 filas con 8 features
y = np.random.rand(1000)        # 1000 etiquetas

## Division del dataset en train (70%), validation (15%), test(15%)

In [50]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=0)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=0)

In [51]:
model = LinearRegression()
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
y_val_pred = model.predict(X_val)
y_test_pred = model.predict(X_test)

train_mse = mean_squared_error(y_train, y_train_pred)
val_mse = mean_squared_error(y_val, y_val_pred)
test_mse = mean_squared_error(y_test, y_test_pred)


In [ ]:
print(f'MSE en entrenamiento: {train_mse}')
print(f'MSE en validación: {val_mse}')
print(f'MSE en test: {test_mse}')